In [1]:
import pandas as pd
import nltk
import pickle
from nltk.tokenize import WordPunctTokenizer
from collections import defaultdict
import json


sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = WordPunctTokenizer()


word_freq = defaultdict(int)

In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import nltk
from nltk.corpus import stopwords
from string import punctuation
import warnings
from langdetect import detect_langs

warnings.filterwarnings("ignore")

df = pd.read_csv('data/fake_or_real_news.csv')

#替换列名
df.rename(columns={"label":"fake"}, inplace=True)
label_map = {"FAKE": 1, "REAL": 0}
df['fake'] = df['fake'].map(label_map)

#丢掉非英文的
def return_lang(x):
    try:
        language = detect_langs(x)[0].lang
    except Exception:
        language = 'other'
    return language
df['language'] = df['text'].apply(return_lang)


df = df[df['language']=='en']

#丢掉无用的列
df = df.drop(['Unnamed: 0','title_vectors','language'],axis=1)

print(df.head())

                                               title  \
0                       You Can Smell Hillary’s Fear   
1  Watch The Exact Moment Paul Ryan Committed Pol...   
2        Kerry to go to Paris in gesture of sympathy   
3  Bernie supporters on Twitter erupt in anger ag...   
4   The Battle of New York: Why This Primary Matters   

                                                text  fake  
0  Daniel Greenfield, a Shillman Journalism Fello...     1  
1  Google Pinterest Digg Linkedin Reddit Stumbleu...     1  
2  U.S. Secretary of State John F. Kerry said Mon...     0  
3  — Kaydee King (@KaydeeKing) November 9, 2016 T...     1  
4  It's primary day in New York and front-runners...     0  


In [3]:
df = df.reset_index(drop=True)

In [4]:
#这里得到doc2vec的每一篇news的vec
import re

words_in_text = []  #create a list called"words_in_text" and save only the text"
for i in range(len(df)): #for each passage, add it to the words_in_text
    words_in_text.append(df.iloc[i]['text'])
temp = ""
for i in range(len(df)):
    temp = (str(words_in_text[i]))
    temp = ''.join([j for j in temp if not j.isdigit()])
    temp = re.sub("[\u0060|\u0021-\u002c|\u002e-\u002f|\u003a-\u003f|\u2200-\u22ff|\uFB00-\uFFFD|\u2E80-\u33FF|\"|\“|\’|\”|-]", '', temp)
    #delete all the punctuation in the passage
    words_in_text[i] = temp
    temp = ""
    
    
import nltk
from nltk.corpus import stopwords
from string import punctuation

stop_words = stopwords.words('english')
#str.translate does lookup in C, faster
mapping_table = {ord(char): u' ' for char in punctuation}
tokenized = [nltk.word_tokenize(line.translate(mapping_table)) for line in words_in_text]

def clean_text(tokenized_list, sw):
    new_list = []
    for doc in tokenized_list:
        new_list.append([token.lower() for token in doc if token.lower() not in sw])
    return new_list

cleaned = clean_text(tokenized, stop_words)

from collections import Counter
#use Counter to count the top 20 frequent words
new_token_counter=Counter(token for sentences in cleaned for token in sentences)
new_top20=new_token_counter.most_common()[:20]
for index, tok in enumerate(new_top20):
    print('{:>10}  freq: {:>7}'.format(tok[0], tok[1]))
    
    

from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

class TagDocIterator:
    def __init__(self, doc_list, idx_list):
        self.doc_list = doc_list
        self.idx_list = idx_list

    def __iter__(self):
        for doc, idx, in zip(self.doc_list, self.idx_list):
            tag = [idx]
            yield TaggedDocument(words=doc, tags=tag)
            
pretrained_emb = "data/glove.6B.100d.txt" 
doc2vec_model = Doc2Vec(size=300, dm = 1, iter=5,pretrained_emb=pretrained_emb)


doc2vec_model.build_vocab(TagDocIterator(cleaned, df.index))
doc2vec_model.train(TagDocIterator(cleaned, df.index), epochs=5, total_examples=doc2vec_model.corpus_count)

       not  freq:   23639
      said  freq:   21173
     trump  freq:   18415
   clinton  freq:   14458
     would  freq:   12666
        us  freq:   12560
    people  freq:   11351
       one  freq:   11198
       new  freq:    9268
         —  freq:    8744
        no  freq:    8559
     state  freq:    8292
      also  freq:    8213
 president  freq:    8149
  campaign  freq:    7460
      like  freq:    7045
    states  freq:    6679
   hillary  freq:    6663
     obama  freq:    6573
     could  freq:    6531


In [5]:
final_feature = []#doc2vec vector for every news
for i in range(len(doc2vec_model.docvecs)):
    final_feature.append(doc2vec_model.docvecs[i])


global doc2vec_count,doc2vec_count_test,is_test
doc2vec_count_train=0
doc2vec_count_test=0
is_test=0

In [ ]:
doc2vec=np.zeros((6295,30,30,300), dtype=np.int32)

for i in range(6295):
    for second in doc2vec[i]:
        for three in second:
            for j in len(three):
                three[j]=final_feature[i][j]


In [3]:
def frequency(x):
    words = word_tokenizer.tokenize(x)
    for word in words:
        word_freq[word] += 1
df['text'].apply(frequency)
print(1)

1


In [4]:
with open('data/word_freq.pickle', 'wb') as g:
    pickle.dump(word_freq, g)
    print(len(word_freq))
    print("word_freq save finished")

84732
word_freq save finished


In [5]:
num_classes = 2
# 将词频排序，并去掉出现次数最小的3个
sort_words = list(sorted(word_freq.items(), key=lambda x:-x[1]))
print(sort_words[:10], sort_words[-10:])

#构建vocablary，并将出现次数小于5的单词全部去除，视为UNKNOW
vocab = {}
i = 1
vocab['UNKNOW_TOKEN'] = 0
for word, freq in word_freq.items():
    if freq > 5:
        vocab[word] = i
        i += 1
print(i)
UNKNOWN = 0

data_x = []
data_y = []
max_sent_in_doc = 30
max_word_in_sent = 30

[('the', 260221), (',', 238341), ('.', 220886), ('to', 138061), ('of', 128907), ('and', 113535), ('a', 103956), ('in', 90130), ('that', 68535), ('’', 56760)] [('471', 1), ('Doanld', 1), ('Folding', 1), ('ACRONYM', 1), ('NCSBM', 1), ('underside', 1), ('suitably', 1), ('Etti', 1), ('monographs', 1), ('bluegrass', 1)]
28551


In [6]:
def set_data(x):
    doc=np.zeros((30,30), dtype=np.int32)
    sents = sent_tokenizer.tokenize(x['text'])
    for i, sent in enumerate(sents):
        if i < max_sent_in_doc:
            word_to_index = []
            for j, word in enumerate(word_tokenizer.tokenize(sent)):
                if j < max_word_in_sent:
                        doc[i][j]=vocab.get(word, UNKNOWN) 

    label = int(x['fake'])
    labels = [0] * num_classes
    labels[label-1] = 1
    data_y.append(labels)
    data_x.append(doc)

df.apply(set_data,axis=1)
    
pickle.dump((data_x, data_y), open('data/fake_news_data', 'wb'))
print(len(data_x))

6295


In [7]:
def read_dataset():
    with open('data/fake_news_data', 'rb') as f:
        data_x, data_y = pickle.load(f)
        length = len(data_x)
        train_x, dev_x = data_x[:int(length*0.7)], data_x[int(length*0.7)+1 :]
        train_y, dev_y = data_y[:int(length*0.7)], data_y[int(length*0.7)+1 :]
        return train_x, train_y, dev_x, dev_y

In [73]:
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.contrib import layers

def length(sequences):
#返回一个序列中每个元素的长度
    used = tf.sign(tf.reduce_max(tf.abs(sequences), reduction_indices=2))
    seq_len = tf.reduce_sum(used, reduction_indices=1)
    return tf.cast(seq_len, tf.int32)

class HAN():

    def __init__(self, vocab_size, num_classes, embedding_size=200, hidden_size=50):

        self.vocab_size = vocab_size
        self.num_classes = num_classes
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size

        with tf.name_scope('placeholder'):
            self.max_sentence_num = tf.placeholder(tf.int32, name='max_sentence_num')
            self.max_sentence_length = tf.placeholder(tf.int32, name='max_sentence_length')
            self.batch_size = tf.placeholder(tf.int32, name='batch_size')
            #x的shape为[batch_size, 句子数， 句子长度(单词个数)]，但是每个样本的数据都不一样，，所以这里指定为空
            #y的shape为[batch_size, num_classes]
            self.input_x = tf.placeholder(tf.int32, [None, None, None], name='input_x')
            self.input_y = tf.placeholder(tf.float32, [None, num_classes], name='input_y')

        #构建模型
        word_embedded = self.word2vec()
        sent_vec = self.sent2vec(word_embedded)
        doc_vec = self.doc2vec(sent_vec)
        out = self.classifer(doc_vec)

        self.out = out
        self.embed_matrix=word_embedded


    def word2vec(self):
        #embedding
        global doc2vec_count_test,doc2vec_count_train,is_test
        with tf.name_scope("embedding"):
            embedding_mat = tf.Variable(tf.truncated_normal((self.vocab_size, self.embedding_size)))
            #shape为[batch_size, sent_in_doc, word_in_sent, embedding_size]
            word_embedded_raw = tf.nn.embedding_lookup(embedding_mat, self.input_x)
            
            
            if is_test==0:
                word_embedded = np.concatenate([final_feature[doc2vec_count_train:doc2vec_count_train+24],word_embedded_raw], axis=0)
                doc2vec_count_train = (doc2vec_count_train + 24)%4392
            else:
                word_embedded = np.concatenate([final_feature[doc2vec_count_test:doc2vec_count_test+24],word_embedded_raw], axis=0)
                doc2vec_count_test = doc2vec_count_test + 24
          
            #for news in word_embedded:
            #    for sentence in news:
            #        for word in sentence:
            #            if is_test==0:
            #                word=final_feature[doc2vec_count_train].append(word)
            #                doc2vec_count_train = (doc2vec_count_train + 1)%4392
            #            else:
            #                word=final_feature[doc2vec_count_test].append(word)
            #                doc2vec_count_train = doc2vec_count_test + 1
            
        return word_embedded

    def sent2vec(self, word_embedded):
        
        with tf.name_scope("sent2vec"):
            #GRU的输入tensor是[batch_size, max_time, ...].在构造句子向量时max_time应该是每个句子的长度，所以这里将
            #batch_size * sent_in_doc当做是batch_size.这样一来，每个GRU的cell处理的都是一个单词的词向量
            #并最终将一句话中的所有单词的词向量融合（Attention）在一起形成句子向量

            
            
            #shape为[batch_size*sent_in_doc, word_in_sent, 300+embedding_size]
            word_embedded = tf.reshape(word_embedded, [-1, self.max_sentence_length, 300+self.embedding_size])
            #shape为[batch_size*sent_in_doce, word_in_sent, hidden_size*2]
            word_encoded = self.BidirectionalGRUEncoder(word_embedded, name='word_encoder')
            #shape为[batch_size*sent_in_doc, hidden_size*2]
            sent_vec = self.AttentionLayer(word_encoded, name='word_attention')
            return sent_vec

    def doc2vec(self, sent_vec):
        #原理与sent2vec一样，根据文档中所有句子的向量构成一个文档向量
        with tf.name_scope("doc2vec"):
            sent_vec = tf.reshape(sent_vec, [-1, self.max_sentence_num, self.hidden_size*2])
            #shape为[batch_size, sent_in_doc, hidden_size*2]
            doc_encoded = self.BidirectionalGRUEncoder(sent_vec, name='sent_encoder')
            #shape为[batch_szie, hidden_szie*2]
            doc_vec = self.AttentionLayer(doc_encoded, name='sent_attention')
            return doc_vec

    def classifer(self, doc_vec):
        #最终的输出层，是一个全连接层
        with tf.name_scope('doc_classification'):
            out = layers.fully_connected(inputs=doc_vec, num_outputs=self.num_classes, activation_fn=None)
            return out

    def BidirectionalGRUEncoder(self, inputs, name):
        #双向GRU的编码层，将一句话中的所有单词或者一个文档中的所有句子向量进行编码得到一个 2×hidden_size的输出向量，然后在经过Attention层，将所有的单词或句子的输出向量加权得到一个最终的句子/文档向量。
        #输入inputs的shape是[batch_size, max_time, voc_size]
        with tf.variable_scope(name):
            GRU_cell_fw = rnn.GRUCell(self.hidden_size)
            GRU_cell_bw = rnn.GRUCell(self.hidden_size)
            #fw_outputs和bw_outputs的size都是[batch_size, max_time, hidden_size]
            ((fw_outputs, bw_outputs), (_, _)) = tf.nn.bidirectional_dynamic_rnn(cell_fw=GRU_cell_fw,
                                                                                 cell_bw=GRU_cell_bw,
                                                                                 inputs=inputs,
                                                                                 sequence_length=length(inputs),
                                                                                 dtype=tf.float32)
            #outputs的size是[batch_size, max_time, hidden_size*2]
            outputs = tf.concat((fw_outputs, bw_outputs), 2)
            return outputs

    def AttentionLayer(self, inputs, name):
        #inputs是GRU的输出，size是[batch_size, max_time, encoder_size(hidden_size * 2)]
        with tf.variable_scope(name):
            # u_context是上下文的重要性向量，用于区分不同单词/句子对于句子/文档的重要程度,
            # 因为使用双向GRU，所以其长度为2×hidden_szie
            u_context = tf.Variable(tf.truncated_normal([self.hidden_size * 2]), name='u_context')
            #使用一个全连接层编码GRU的输出的到期隐层表示,输出u的size是[batch_size, max_time, hidden_size * 2]
            h = layers.fully_connected(inputs, self.hidden_size * 2, activation_fn=tf.nn.tanh)
            #shape为[batch_size, max_time, 1]
            alpha = tf.nn.softmax(tf.reduce_sum(tf.multiply(h, u_context), axis=2, keep_dims=True), dim=1)
            #reduce_sum之前shape为[batch_szie, max_time, hidden_szie*2]，之后shape为[batch_size, hidden_size*2]
            atten_output = tf.reduce_sum(tf.multiply(inputs, alpha), axis=1)
            return atten_output

In [9]:
import tensorflow as tf
import time
import os


# Data loading params
tf.flags.DEFINE_integer("vocab_size", 84729, "vocabulary size")
tf.flags.DEFINE_integer("num_classes", 2, "number of classes")
tf.flags.DEFINE_integer("embedding_size", 300, "Dimensionality of character embedding (default: 200)")
tf.flags.DEFINE_integer("hidden_size", 50, "Dimensionality of GRU hidden layer (default: 50)")
tf.flags.DEFINE_integer("batch_size", 24, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 10, "Number of training epochs (default: 50)")
tf.flags.DEFINE_integer("checkpoint_every", 50, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
tf.flags.DEFINE_integer("evaluate_every", 50, "evaluate every this many batches")
tf.flags.DEFINE_float("learning_rate", 0.01, "learning rate")
tf.flags.DEFINE_float("grad_clip", 5, "grad clip to prevent gradient explode")

tf.app.flags.DEFINE_string('f', '', 'kernel')

FLAGS = tf.flags.FLAGS

train_x, train_y, dev_x, dev_y = read_dataset()
print("data load finished")

data load finished


In [74]:
tf.reset_default_graph()

sess = tf.InteractiveSession()

han = HAN(vocab_size=FLAGS.vocab_size,
                    num_classes=FLAGS.num_classes,
                    embedding_size=FLAGS.embedding_size,
                    hidden_size=FLAGS.hidden_size)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=han.input_y,
                                                              logits=han.out,
                                                              name='loss'))
embed_matrix=han.embed_matrix
output_vec=han.out
input_label=han.input_y
predict = tf.argmax(han.out, axis=1, name='predict')
label = tf.argmax(han.input_y, axis=1, name='label')
acc = tf.reduce_mean(tf.cast(tf.equal(predict, label), tf.float32))
    
global_step = tf.Variable(0, trainable=False)
optimizer = tf.train.AdamOptimizer(FLAGS.learning_rate)
# RNN中常用的梯度截断，防止出现梯度过大难以求导的现象
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), FLAGS.grad_clip)
grads_and_vars = tuple(zip(grads, tvars))
train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())



def train_step(x_batch, y_batch):
    feed_dict = {
        han.input_x: x_batch,
        han.input_y: y_batch,
        han.max_sentence_num: 30,
        han.max_sentence_length: 30,
        han.batch_size: 24
    }
    _, step, cost, accuracy, embed = sess.run([train_op, global_step , loss, acc, embed_matrix], feed_dict)
    time_str = str(int(time.time()))
    if step==1:
        print(len(embed))
        print(len(embed[0]))
        print(len(embed[0][0]))
        print(len(embed[0][0][0]))
    
    if step%20==0:
        print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, cost, accuracy))
        

    return step

def dev_step(x_batch, y_batch, writer=None):
    feed_dict = {
        han.input_x: x_batch,
        han.input_y: y_batch,
        han.max_sentence_num: 30,
        han.max_sentence_length: 30,
        han.batch_size: 24
    }
    step, cost, accuracy = sess.run([global_step,  loss, acc], feed_dict)
    time_str = str(int(time.time()))
    print("++++++++++++++++++dev++++++++++++++{}: step {}, loss {:g}, acc {:g}".format(time_str, step, cost, accuracy))

save_version=0
for epoch in range(FLAGS.num_epochs):
    print('current epoch %s' % (epoch + 1))
    for i in range(0, 4392, FLAGS.batch_size):
        x = train_x[i:i + FLAGS.batch_size]
        y = train_y[i:i + FLAGS.batch_size]
        step = train_step(x, y)
        if step % FLAGS.evaluate_every == 0:
            dev_step(dev_x, dev_y)
            sess.run(tf.Print(embed_matrix,[embed_matrix]))
            save_path = saver.save(sess, "model/pretrained_lstm.ckpt", global_step=save_version)
            print("saved to %s" % save_path)
            save_version+=1

current epoch 1
24
30
30
300


KeyboardInterrupt: 

In [39]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('model'))

predict_value=[]
right_label=[]


def test_step(x_batch, y_batch, writer=None):
    feed_dict = {
        han.input_x: x_batch,
        han.input_y: y_batch,
        han.max_sentence_num: 30,
        han.max_sentence_length: 30,
        han.batch_size: 24
    }
    cost, accuracy, predict, label = sess.run([loss, acc, output_vec, input_label], feed_dict)
    predict_value.append(predict)
    right_label.append(label)
    print("loss {:g}, acc {:g}".format(cost, accuracy))

for i in range(0, 1772, FLAGS.batch_size):
    x = dev_x[i:i + FLAGS.batch_size]
    y = dev_y[i:i + FLAGS.batch_size]
    step = test_step(x, y)

INFO:tensorflow:Restoring parameters from model\pretrained_lstm.ckpt-35
loss 0.21478, acc 0.958333
loss 0.262696, acc 0.875
loss 0.0035098, acc 1
loss 0.159963, acc 0.958333
loss 0.000259335, acc 1
loss 0.389061, acc 0.916667
loss 0.624741, acc 0.916667
loss 0.399451, acc 0.875
loss 0.304654, acc 0.958333
loss 0.311004, acc 0.916667
loss 0.259375, acc 0.958333
loss 0.0144408, acc 1
loss 0.594512, acc 0.958333
loss 0.0314883, acc 0.958333
loss 0.00810966, acc 1
loss 0.0804162, acc 0.958333
loss 0.0192032, acc 1
loss 0.74801, acc 0.916667
loss 0.462742, acc 0.916667
loss 0.227661, acc 0.958333
loss 0.01282, acc 1
loss 0.0790868, acc 0.958333
loss 0.584973, acc 0.958333
loss 0.364194, acc 0.833333
loss 4.38152e-05, acc 1
loss 0.141635, acc 0.958333
loss 0.168854, acc 0.958333
loss 0.109422, acc 0.958333
loss 0.085621, acc 0.958333
loss 0.356664, acc 0.958333
loss 0.00141188, acc 1
loss 0.00119072, acc 1
loss 0.490177, acc 0.958333
loss 0.0666249, acc 0.958333
loss 0.0180926, acc 1
loss 0.

In [42]:
predict_value

[array([[-4.37506056,  2.31439805],
        [-4.20665789,  3.96711588],
        [-6.87247944,  5.1461606 ],
        [ 7.28684664, -5.15196085],
        [ 7.10223341, -6.26463747],
        [ 0.67502868, -1.37904882],
        [ 7.9846139 , -6.1807723 ],
        [ 6.06219912, -5.43939924],
        [-7.55500507,  5.56114244],
        [ 4.81872797, -3.2431376 ],
        [-6.17405367,  4.25289202],
        [-7.21705294,  5.29296446],
        [ 3.88542295, -3.65764832],
        [ 6.68246555, -6.04409313],
        [ 5.92433548, -4.42574358],
        [ 6.91344643, -5.26543856],
        [-2.97908831,  2.04309082],
        [-3.6599679 ,  2.32475305],
        [ 4.33460188, -3.72785139],
        [-7.28916407,  5.44590425],
        [-7.87186289,  5.59841681],
        [ 8.89468861, -6.58637333],
        [-5.57471132,  5.82254744],
        [-6.58808661,  4.44924307]], dtype=float32),
 array([[-8.26143169,  5.91990995],
        [ 0.3795684 , -0.41022936],
        [ 6.45554209, -5.5109067 ],
        [ 4

In [46]:
data_x2 = []
data_y2 = []
max_sent_in_doc = 30
max_word_in_sent = 30

In [47]:
def set_data2(x):
    doc=np.zeros((30,30), dtype=np.int32)
    sents = sent_tokenizer.tokenize(x['text'])
    for i, sent in enumerate(sents):
        i_verse = len(sents)-1-i   
        if  i_verse < max_sent_in_doc:
            for j, word in enumerate(word_tokenizer.tokenize(sent)):
                if j < max_word_in_sent:
                        doc[29-i_verse][j]=vocab.get(word, UNKNOWN) 

    label = int(x['fake'])
    labels = [0] * num_classes
    labels[label-1] = 1
    data_y2.append(labels)
    data_x2.append(doc)

df.apply(set_data2,axis=1)
    
pickle.dump((data_x2, data_y2), open('data/fake_news_data2', 'wb'))
print(len(data_x2))

6295


In [48]:
def read_dataset2():
    with open('data/fake_news_data2', 'rb') as f:
        data_x2, data_y2= pickle.load(f)
        length = len(data_x2)
        train_x2, dev_x2 = data_x2[:int(length*0.7)], data_x2[int(length*0.7)+1 :]
        train_y2, dev_y2 = data_y2[:int(length*0.7)], data_y2[int(length*0.7)+1 :]
        return train_x2, train_y2, dev_x2, dev_y2

In [ ]:
train_x2, train_y2, dev_x2, dev_y2 = read_dataset2()
print("data2 load finished")


tf.reset_default_graph()

sess = tf.InteractiveSession()

han = HAN(vocab_size=FLAGS.vocab_size,
                    num_classes=FLAGS.num_classes,
                    embedding_size=FLAGS.embedding_size,
                    hidden_size=FLAGS.hidden_size)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=han.input_y,
                                                              logits=han.out,
                                                              name='loss'))

output_vec=han.out
input_label=han.input_y
predict = tf.argmax(han.out, axis=1, name='predict')
label = tf.argmax(han.input_y, axis=1, name='label')
acc = tf.reduce_mean(tf.cast(tf.equal(predict, label), tf.float32))
    
global_step = tf.Variable(0, trainable=False)
optimizer = tf.train.AdamOptimizer(FLAGS.learning_rate)
# RNN中常用的梯度截断，防止出现梯度过大难以求导的现象
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), FLAGS.grad_clip)
grads_and_vars = tuple(zip(grads, tvars))
train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())



def train_step(x_batch, y_batch):
    feed_dict = {
        han.input_x: x_batch,
        han.input_y: y_batch,
        han.max_sentence_num: 30,
        han.max_sentence_length: 30,
        han.batch_size: 24
    }
    _, step, cost, accuracy = sess.run([train_op, global_step , loss, acc], feed_dict)
    
    time_str = str(int(time.time()))
    print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, cost, accuracy))

    return step

def dev_step(x_batch, y_batch, writer=None):
    feed_dict = {
        han.input_x: x_batch,
        han.input_y: y_batch,
        han.max_sentence_num: 30,
        han.max_sentence_length: 30,
        han.batch_size: 24
    }
    step, cost, accuracy = sess.run([global_step,  loss, acc], feed_dict)
    time_str = str(int(time.time()))
    print("++++++++++++++++++dev++++++++++++++{}: step {}, loss {:g}, acc {:g}".format(time_str, step, cost, accuracy))

save_version=0
for epoch in range(FLAGS.num_epochs):
    print('current epoch %s' % (epoch + 1))
    for i in range(0, 4392, FLAGS.batch_size):
        x = train_x2[i:i + FLAGS.batch_size]
        y = train_y2[i:i + FLAGS.batch_size]
        step = train_step(x, y)
        if step % FLAGS.evaluate_every == 0:
            dev_step(dev_x2, dev_y2)
        
            save_path = saver.save(sess, "model/text2/pretrained_lstm.ckpt", global_step=save_version)
            print("saved to %s" % save_path)
            save_version+=1

data2 load finished
current epoch 1
